In [ ]:
# !curl -X GET "nersc-tbn-6.testbed100.es.net:9200/iperf3*/_search" | jq 

In [ ]:
'''
**
** Project Lead: Eashan Adhikarla
** Mentor: Ezra Kissel
** 
** Date Created: June 17' 2021
** Last Modified: June 22' 2021 
**
'''

from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan

import pandas as pd
import requests
import json

# Create the elasticsearch client
HOST = 'nersc-tbn-6.testbed100.es.net'
PORT = 9200

es = Elasticsearch(host=HOST, port=PORT, timeout=50)


class clr:
    """
    Defining colors for the print syntax coloring
    """
    H   = '\033[35m' # Header
    B   = '\033[94m' # Blue
    G   = '\033[36m' # Green
    W   = '\033[93m' # Warning
    F   = '\033[91m' # Fail
    E   = '\033[0m'  # End 
    BD  = '\033[1m'  # Bold
    UL  = '\033[4m'  # Underline


class GETTER:
    """
    Get class to get current information about different Indexs
    """
    def __init__(self, term, sum=0):
        self.term = term
        self.sum = sum

    def getIndexList(self, term):
        idx=[]
        indices_dict = es.indices.get_alias(self.term)
        if isinstance(indices_dict, dict) and indices_dict is not None:
            print(f"'{clr.G}{len(indices_dict)}{clr.E}' indexes found!")
            for k,v in indices_dict.items():
                idx.append(k)
            return idx
        else:
            print (f"{clr.F}Empty dict!{clr.E}")


    def getIndexedDetails(self, indexes):
        for i in range(1): # len(indexes)):
            try:
                # Given a index name, finds all the documents in the index
                # index: Index name as a string
                # body: Empty equivalent to '*' to get all the values
                # size: To some random max value for getting all the documents
                result = es.search(index=indexes[i],
                                body={"query":{"match_all":{}}},
                                size=1000,
                                )
                print(f"{indexes[i]} ---> {clr.G}{result['hits']['total']['value']}{clr.E} documents\n")
                
                documents = [doc for doc in result['hits']['hits']]
                
                for j in range(len(documents)):
                    # ---------------------
                    # For each job/document
                    # ---------------------
                    # Format: dict_keys(['end', '@version', 'start', 'uuid', 'intervals', '@timestamp'])
                    # uuid
                    uuid = documents[j]['_source']['uuid']
                    # @timestamp
                    timestamp = documents[j]['_source']['@timestamp']
                    # start (Format: dict_keys(['cookie', 'test_start', 'tcp_mss_default', 'version', 'connected', 'sndbuf_actual',
                    #                           'rcvbuf_actual', 'sock_bufsize', 'system_info', 'timestamp', 'connecting_to']))
                    start_dict = documents[j]['_source']['start']
                    num_streams = start_dict['test_start']['num_streams']

                    # Congestion Type
                    sender_congestion = documents[j]['_source']['end']['sender_tcp_congestion']
                    receiver_congestion = documents[j]['_source']['end']['receiver_tcp_congestion']


                    # print(f"uuid: {uuid}\ntimestamp: {timestamp}\nnum_streams: {num_streams}\n\n")

                    # --------------------- 
                    # For each stream/flow
                    # ---------------------
                    # Intervals
                    intervals_list = documents[j]['_source']['intervals']
                    for k in range(len(intervals_list)):
                        for m in range(num_streams):
                            # Format: dict_keys(['receiver', 'sender'])
                            # 'sender': dict_keys(['retransmits', 'max_rtt', 'sender', 'start', 'bytes', 'mean_rtt', 'end'
                            #                      'max_snd_cwnd', 'bits_per_second', 'socket', 'seconds', 'min_rtt'])
                            # 'receiver': dict_keys(['end', 'bits_per_second', 'sender', 'start', 'socket', 'seconds', 'bytes'])
                            
                            sender_start = documents[j]['_source']['end']['streams'][m]['sender']['start']
                            sender_end = documents[j]['_source']['end']['streams'][m]['sender']['end']
                            sender_retransmits = documents[j]['_source']['end']['streams'][m]['sender']['retransmits']
                            sender_bytes = documents[j]['_source']['end']['streams'][m]['sender']['bytes']
                            sender_min_rtt = documents[j]['_source']['end']['streams'][m]['sender']['min_rtt']
                            sender_max_rtt = documents[j]['_source']['end']['streams'][m]['sender']['max_rtt']
                            sender_mean_rtt = documents[j]['_source']['end']['streams'][m]['sender']['mean_rtt']
                            sender_bps = documents[j]['_source']['end']['streams'][m]['sender']['bits_per_second']

                            receiver_start = documents[j]['_source']['end']['streams'][m]['receiver']['start']
                            receiver_end = documents[j]['_source']['end']['streams'][m]['receiver']['end']
                            receiver_seconds = documents[j]['_source']['end']['streams'][m]['receiver']['seconds']
                            receiver_bytes = documents[j]['_source']['end']['streams'][m]['receiver']['bytes']
                            receiver_bps = documents[j]['_source']['end']['streams'][m]['receiver']['bits_per_second']


                            print(f"uuid: {uuid}\ntimestamp: {timestamp}\nnum_streams: {num_streams}\nsender_congestion: {sender_congestion}\nreceiver_congestion: {receiver_congestion}\nsender_start: {sender_start}\n sender_end: {sender_end}\n sender_retransmits: {sender_retransmits}\nsender_bytes: {sender_bytes}\n sender_min_rtt: {sender_min_rtt}\n sender_max_rtt: {sender_max_rtt}\nsender_mean_rtt: {sender_mean_rtt}\n sender_bps: {sender_bps}\n receiver_start: {receiver_start}\nreceiver_end: {receiver_end}\n receiver_seconds: {receiver_seconds}\n receiver_bytes: {receiver_bytes}\nreceiver_bps: {receiver_bps}\n\n")

            except:
                pass
            # print("\nTotal docs found: ", self.sum)
        
        return None

indexTypes = ["*", "iperf*", "jobmeta*", "bbrmon*"]
term_ = indexTypes[0]

get = GETTER(term_)
indexes = get.getIndexList(term_)
for e,i in enumerate(indexes):
    print(f"{e}: {i}")

# index_response = get.getIndexedDetails(indexes)


In [ ]:
sorted_ind = sorted(indexes)
sorted_ind

In [ ]:
iperf3 = [i for i in sorted_ind if i.startswith("iperf3")]
jobmeta = [i for i in sorted_ind if i.startswith("jobmeta")]
ss = [i for i in sorted_ind if i.startswith("ss")]
bbrmon_pscheduler = [i for i in sorted_ind if i.startswith("bbrmon-pscheduler")]
bbrmon_jobmeta = [i for i in sorted_ind if i.startswith("bbrmon-jobmeta")]
bbrmon_ss = [i for i in sorted_ind if i.startswith("bbrmon-ss")]
bbrmon_tcptrace = [i for i in sorted_ind if i.startswith("bbrmon-tcptrace")]
miscellaneous = [i for i in sorted_ind if i not in (iperf3+jobmeta+ss+bbrmon_jobmeta+bbrmon_pscheduler+bbrmon_ss+bbrmon_tcptrace)]

In [ ]:
len(iperf3+jobmeta+ss+bbrmon_jobmeta+bbrmon_pscheduler+bbrmon_ss+bbrmon_tcptrace+miscellaneous) == len(sorted_ind)

In [ ]:
i=0
result = es.search(index=jobmeta[i],
                                body={"query":{"match_all":{}}},
                                size=10000,
                                )
data = [doc for doc in result['hits']['hits']]

In [ ]:
data[0].keys()

In [ ]:
result['hits'].keys()

In [ ]:
print(result['hits']['total'])
print(result['hits']['max_score'])

In [ ]:
result['hits']['hits'][0].keys()

In [ ]:
print(result['hits']['hits'][0]['_index']) #.keys()
print(result['hits']['hits'][0]['_type'])
print(result['hits']['hits'][0]['_id'])
print(result['hits']['hits'][0]['_score'])

In [ ]:
result['hits']['hits'][0]['_source'].keys()

In [ ]:
result['hits']['hits'][0]['_source']['iter_uuids']

In [ ]:
result['hits']['hits'][0]['_source']['end'].keys()

In [ ]:
result['hits']['hits'][0]['_source']['@version']

In [ ]:
result['hits']['hits'][0]['_source']['start']

In [ ]:
result['hits']['hits'][0]['_source']['uuid']

In [ ]:
result['hits']['hits'][0]['_source']['intervals'][0]

In [ ]:
result['hits']['hits'][0]['_source']['@timestamp']

In [ ]:
# for i in range(len(data)):
#     print(len(data[i]['_source']['end']['streams'])) # .keys()#['bits_per_second'] # [].keys()
# data[0]['_source']['end']['streams'][0] #['sender'].keys()

x = data[0]['_source']['end']['streams'][:]
# x['sender']['start']
a[:] = x[:]['sender']['start']

In [ ]:
# es.get(index="iperf3-2021.06.06", id="uhKs33kBkImc33Nl6K8P")

In [ ]:
# result = es.search(index=indexes[0], body={"query":{"match_all":{}}})
# indexes[0]

In [ ]:
result = es.search(index=indexes[0], body={"query":{"match_all":{}}})

# hitsList=[]
# print(f"Found {len(result.items())} keys found..\n")

print(f"Found '{clr.B}{clr.BD}{result['hits']['total']['value']}{clr.E}{clr.E}' documents in the current index'{clr.B}{clr.BD}{indexes[0]}{clr.E}{clr.E}'")

data = [doc for doc in result['hits']['hits']]

i=0
print(data[i]['_source'].keys())
print("\n")
print(data[i]['_source']['end'])
# print(data[i]['_source']['@version'])
# print(data[i]['_source']['start'])
# print(data[i]['_source']['uuid'])
# print(data[i]['_source']['@timestamp'])
print()

i=1
print(data[i]['_source'].keys())
print("\n")
print(data[i]['_source']['end'])
# print(data[i]['_source']['@version'])
# print(data[i]['_source']['start'])
# print(data[i]['_source']['uuid'])
# print(data[i]['_source']['@timestamp'])
print()

i=2
print(data[i]['_source'].keys())
print("\n")
print(data[i]['_source']['end'])
# print(data[i]['_source']['@version'])
# print(data[i]['_source']['start'])
# print(data[i]['_source']['uuid'])
# print(data[i]['_source']['@timestamp'])
print()

# for doc in data:
#     print(doc['_source'].keys())
#     print(doc['_source']['start'])
#     print(doc['_source']['uuid'])
#     print(doc['_source']['@timestamp'])
#     break
    # print(f"{doc['_id']}, {doc['_source']}"), {doc['_source']}" )

In [ ]:
# '''
# **
# ** Project Lead: Eashan Adhikarla
# ** Mentor: Ezra Kissel
# ** 
# ** Date Created: June 17' 2021
# ** Last Modified: June 22' 2021 
# **
# '''

# from elasticsearch import Elasticsearch
# from elasticsearch.helpers import scan

# import pandas as pd
# import requests
# import json

# # Create the elasticsearch client
# HOST = 'nersc-tbn-6.testbed100.es.net'
# PORT = 9200

# es = Elasticsearch(host=HOST, port=PORT)


# class clr:
#     """
#     Defining colors for the print syntax coloring
#     """
#     H   = '\033[35m' # Header
#     B   = '\033[94m' # Blue
#     G   = '\033[36m' # Green
#     W   = '\033[93m' # Warning
#     F   = '\033[91m' # Fail
#     E   = '\033[0m'  # End 
#     BD  = '\033[1m'  # Bold
#     UL  = '\033[4m'  # Underline


# class GETTER:
#     """
#     Get class to get current information about different Indexs
#     """
#     def __init__(self, term, sum=0):
#         self.term = term
#         self.sum = sum

#     def getIndexList(self, term):
#         idx=[]
#         indices_dict = es.indices.get_alias(self.term)
#         if isinstance(indices_dict, dict) and indices_dict is not None:
#             print(f"'{clr.G}{len(indices_dict)}{clr.E}' indexes found!")
#             for k,v in indices_dict.items():
#                 idx.append(k)
#             return idx
#         else:
#             print (f"{clr.F}Empty dict!{clr.E}")


#     def getIndexedDetails(self, indexes):
#         for i in range(1): # len(indexes)):
#             try:
#                 # Given a index name, finds all the documents in the index
#                 # index: Index name as a string
#                 # body: Empty equivalent to '*' to get all the values
#                 # size: To some random max value for getting all the documents
#                 result = es.search(index=indexes[i],
#                                 body={"query":{"match_all":{}}},
#                                 size=10000,
#                                 )
#                 print(f"{indexes[i]} ---> {clr.G}{result['hits']['total']['value']}{clr.E} documents\n")
                
#                 documents = [doc for doc in result['hits']['hits']]
                
#                 for j in range(len(documents)):
#                     # ---------------------
#                     # For each job/document
#                     # ---------------------
#                     # Format: dict_keys(['end', '@version', 'start', 'uuid', 'intervals', '@timestamp'])
#                     # uuid
#                     uuid = documents[j]['_source']['uuid']
#                     # @timestamp
#                     timestamp = documents[j]['_source']['@timestamp']
#                     # start (Format: dict_keys(['cookie', 'test_start', 'tcp_mss_default', 'version', 'connected', 'sndbuf_actual',
#                     #                           'rcvbuf_actual', 'sock_bufsize', 'system_info', 'timestamp', 'connecting_to']))
#                     start_dict = documents[j]['_source']['start']
#                     num_streams = start_dict['test_start']['num_streams']

#                     # --------------------- 
#                     # For each stream/flow
#                     # ---------------------
#                     # Intervals
#                     intervals_list = documents[j]['_source']['intervals']
#                     for k in range(len(intervals_list)):
#                         for m in range(num_streams):
#                             # Format: dict_keys(['receiver', 'sender'])
#                             # 'receiver': dict_keys(['end', 'bits_per_second', 'sender', 'start', 'socket', 'seconds', 'bytes'])
#                             # 'sender': dict_keys(['retransmits', 'max_rtt', 'sender', 'start', 'bytes', 'mean_rtt', 'end'
#                             #                      'max_snd_cwnd', 'bits_per_second', 'socket', 'seconds', 'min_rtt'])
#                             data[0]['_source']['end']['streams']
                            
#                             # Format: dict_keys(['retransmits', 'omitted', 'start', 'bytes', 'snd_cwnd', 'pmtu', 'rttvar', 'sender', 'end', 'bits_per_second', 'socket', 'seconds', 'rtt'])
#                             # retransmits = documents[j]['_source']['intervals'][k]['streams'][m]['retransmits']
#                             # rttvar      = documents[j]['_source']['intervals'][k]['streams'][m]['rttvar']
#                             # rtt         = documents[j]['_source']['intervals'][k]['streams'][m]['rtt']
#                             # bits_per_second = documents[j]['_source']['intervals'][k]['streams'][m]['bits_per_second']
                    
#                     print(f"uuid: {uuid}\ntimestamp: {timestamp}\nnum_streams: {num_streams}\nretransmits: {retransmits}\nrttvar: {rttvar}\nrtt: {rtt}\nbps: {bits_per_second}\n\n")



                    
                
#                 # self.sum += result['hits']['total']['value']
                
#                 # # iterate the nested dictionaries inside the ["hits"]["hits"] list
#                 # for num, doc in enumerate(data):
#                 #     # print ("DOC ID:", doc["_id"], "--->", doc, type(doc), "\n")
                    
#                 #     # Use 'iteritems()` instead of 'items()' if using Python 2
#                 #     for key, value in doc.items():
#                 #         print (f"{clr.B}{key}{clr.E}   ---> {value}")

#                 #     # print a few spaces between each doc for readability
#                 #     print ("\n\n")
#                 #     break
#             except:
#                 pass
#             # print("\nTotal docs found: ", self.sum)
        
#         return None

# indexTypes = ["*", "iperf*", "jobmeta*", "bbrmon*"]
# term_ = indexTypes[1]

# get = GETTER(term_)
# indexes = get.getIndexList(term_)
# # for e,i in enumerate(indexes):
# #     print(f"{e}: {i}")

# index_response = get.getIndexedDetails(indexes)



In [ ]:
for k2,v2 in hitsList[0][0].items():
    print(k2)
    if k2=="_score":
        for v in value:
            print(v)

In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch(host='nersc-tbn-6.testbed100.es.net', port=9200)
res = es.search(index="iperf3*",
                # doc_type="articles",
                body={"query": 
                        {"match": 
                                {"content": ""}
                        }
                      })
print("res: ", res)

print(f"{res['hits']['total']} documents found")

for doc in res['hits']['hits']:
    print("%s) %s" % (doc['_id'], doc['_source']['content']))


## Querying elasticsearch via elasticsearch-py module

In [ ]:
# def get_data_from_elastic():
    
# query: The elasticsearch query.
query = {
    "query": {
        "match": {
            "symbols.keyword": ""
        }
    }
}

# Scan function to get all the data. 
rel = scan( client=es,             
            query=query,                                     
            scroll='1m',
            index='iperf3*',
            raise_on_error=True,
            preserve_order=True,
            clear_scroll=True)

# Keep response in a list.
if not rel==None:
    result = list(rel)

print(result)

# temp = []

# # We need only '_source', which has all the fields required.
# # This elimantes the elasticsearch metdata like _id, _type, _index.
# for hit in result:
#     temp.append(hit['_source'])

# # Create a dataframe.
# df = pd.DataFrame(temp)

# return df


## Querying elasticsearch via REST api

In [ ]:
def search(uri, term):
    """
    Simple Elasticsearch Query
    """
    query = json.dumps({
        "query": {
            "match": {
                "content": term
            }
        }
    })
    response = requests.get(uri, data=query)
    results = json.loads(response.text)
    return results

def format_results(results):
    """
    Print results nicely:
    doc_id) content
    """
    data = [doc for doc in results['hits']['hits']]
    for doc in data:
        print("%s) %s" % (doc['_id'], doc['_source']['content']))

def create_doc(uri, doc_data={}):
    """
    Create new document.
    """
    query = json.dumps(doc_data)
    response = requests.post(uri, data=query)
    print(response)

In [ ]:
# df = get_data_from_elastic()
# print(df.head())

In [ ]:
if __name__ == '__main__':
    uri_search = 'http://192.168.120.46:9200/'
    # uri_search = 'http://192.168.120.46:9200/test/articles/_search'
    # uri_create = 'http://localhost:9200/test/articles/'

    results = search(uri_search, "_source")
    print(results)
    # format_results(results)

    #create_doc(uri_create, {"content": "The fox!"})
    #results = search(uri_search, "fox")
    #format_results(results)

In [ ]:
# !curl -XGET 192.168.120.46:9200/iperf3*/_search | jq >> sample.txt

In [ ]:
import subprocess

list_files = subprocess.run(["ls", "-al"])
print("The exit code was: %d" % list_files.returncode)

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan

# Create the elasticsearch client
HOST = 'nersc-tbn-6.testbed100.es.net'
PORT = 9200

es = Elasticsearch(host=HOST, port=PORT, timeout=30)

# all = es.indices.get_alias("*").keys()

jobmeta = es.indices.get_alias("jobmeta-**").keys()
iperf3 = es.indices.get_alias("iperf3*").keys()
bbrmon_pscheduler = es.indices.get_alias("bbrmon-pscheduler*").keys()
bbrmon_jobmeta = es.indices.get_alias("bbrmon-jobmeta*").keys()

# print(len(jobmeta)), len(iperf3))
# print(jobmeta, iperf3)
print( len(bbrmon_pscheduler), len(bbrmon_jobmeta) )

# iperf3 = sorted(iperf3)
# jobmeta = sorted(jobmeta)
bbrmon_pscheduler = sorted(bbrmon_pscheduler)
bbrmon_jobmeta = sorted(bbrmon_jobmeta)

In [ ]:
jobmeta = jobmeta[5:]
for j in jobmeta:
    print(j)

In [ ]:
key = 'start_time'

for j in jobmeta:
    print("------>", j)
    jobmeta_result = es.search( index=j,
                                body={"query":{"match_all":{}}},
                                size=10000,
                                )
    print(len(jobmeta_result))
    for i in range(len(jobmeta_result)):
        try:
            print(jobmeta_result['hits']['hits'][i]['_source']['pacing'])
            print(jobmeta_result['hits']['hits'][i]['_source']['alias'])
        except:
            print(jobmeta_result['hits']['hits'][i]['_source']['alias'])
        print(jobmeta_result['hits']['hits'][i]['_source'].keys())
        print()

In [ ]:
for i in range(len(jobmeta_result)):
    key = 'start_time'
    # print(jobmeta_result['hits']['hits'][i]['_source'].keys())
    if key in jobmeta_result['hits']['hits'][i].keys():
        print(jobmeta_result['hits']['hits'][i]['_source']['start_time'])

In [ ]:
jobmeta_result['hits']['hits'][0]['_source'].keys() #['alias']

In [ ]:
for x in jobmeta:
    jobmeta_result = es.search( index=x,
                                body={"query":{"match_all":{}}},
                                size=1000,
                                )
    jobmeta_documents = [doc for doc in jobmeta_result['hits']['hits']]
    print(f"{len(jobmeta_documents)} found in index {x}")

    for i in iperf3:
        iperf3_results = es.search( index=i,
                                    body={"query":{"match_all":{}}},
                                    size=1000,
                                   )
        iperf3_documents = [doc for doc in iperf3_results['hits']['hits']]
        for doc in iperf3_documents:
            uuid = doc['_source']['uuid']
            for jobdoc in jobmeta_documents:
                try:
                    iter_uuids = jobdoc["_source"]["iter_uuids"]
                except:
                    pass
                if uuid in iter_uuids:
                    timestamp = doc['_source']['@timestamp']
                    start_dict = doc['_source']['start']
                    num_streams = start_dict['test_start']['num_streams']

                    print("uuid: ", uuid)
                    print("timestamp: ", timestamp)
                    print("num_streams: ", num_streams)
                    print()

In [ ]:
jobdoc['_source']['hostname']#.keys()

In [ ]:
import re, datetime

for a in all:
    try:
        match = re.search('\d{4}.\d{2}.\d{2}', a)
        date = datetime.datetime.strptime(match.group(), '%Y.%m.%d').date()
        # print(f"{a}  --->   {date}")
    except:
        pass
        # print(f"{a}  ---->   Not found!")

In [ ]:
from_date = datetime.datetime(date.year, date.month, date.day)
curr = datetime.datetime.now()
to_date = datetime.datetime(curr.year, curr.month, curr.day)

print(from_date, to_date)

In [ ]:
date = datetime.datetime(2021,5,15)
from_date < date < to_date

In [ ]:
import datetime
# curr = datetime.date.today()
curr = datetime.datetime.now()


In [ ]:
f"{curr.year}-{curr.month}-{curr.day}"

In [ ]:
iperf3[5]

In [ ]:
iperf3_result = es.search( index=iperf3[5],
                            body={"query":{"match_all":{}}},
                            size=10000,
                            )
iperf3_doc = [doc for doc in iperf3_result['hits']['hits']]
print(len(iperf3_doc))

In [ ]:
len(iperf3_doc)
iperf3_doc[0]['_source'].keys()

In [ ]:
print(iperf3_doc[0]['_source']['end'].keys())
print(iperf3_doc[0]['_source']['start'].keys())

In [ ]:
print(iperf3_doc[0]['_source']['intervals'][0].keys())

In [ ]:
iperf3_doc[0]['_source']['end']['streams']

In [ ]:
iperf3_doc[0]['_source']['end']['streams'][0]['sender']

In [ ]:
for i in range(len(bbrmon_jobmeta)):
    try:
        bbrmon_jobmeta_result = es.search(index=bbrmon_jobmeta[i], 
                                        body={"query":{"match_all":{}}},
                                        size=10000)
        # bbrmon_jobmeta_documents = [doc for doc in bbrmon_jobmeta_result['hits']['hits']]
        print("Index: ",i)
        print(bbrmon_jobmeta_result)
    except:
        pass

In [37]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan

# Create the elasticsearch client
HOST = 'nersc-tbn-6.testbed100.es.net'
PORT = 9200

es = Elasticsearch(host=HOST, port=PORT, timeout=30)

bbrmon_pscheduler = es.indices.get_alias("bbrmon-pscheduler*").keys()
bbrmon_jobmeta = es.indices.get_alias("bbrmon-jobmeta*").keys()
bbrmon_ss = es.indices.get_alias("bbrmon-ss*").keys()

bbrmon_pscheduler = sorted(bbrmon_pscheduler)
bbrmon_jobmeta = sorted(bbrmon_jobmeta)
bbrmon_ss = sorted(bbrmon_ss)

print( len(bbrmon_pscheduler), len(bbrmon_jobmeta), len(bbrmon_ss) )

164 54 69


In [6]:
# for i in range(len()):
# try:
#     bbrmon_jobmeta_result = es.search(index=bbrmon_jobmeta[0],
#                                     body={"query":{"match_all":{}}},
#                                     size=10000,
#                                     )
#     bbrmon_jobmeta_result

bbrmon_pscheduler_result = es.search(index=bbrmon_pscheduler[0],
                                body={"query":{"match_all":{}}},
                                size=10000,
                                )
bbrmon_pscheduler_documents = [doc for doc in bbrmon_pscheduler_result['hits']['hits']]

bbrmon_pscheduler_documents[0].keys()



dict_keys(['_index', '_type', '_id', '_score', '_source'])

In [9]:
timestamp = bbrmon_pscheduler_documents[0]['_source']['@timestamp']#.keys()

'2021-02-11T19:05:08.972Z'

In [13]:
bbrmon_pscheduler_documents[0]['_source'].keys()

dict_keys(['@timestamp', 'test', 'tags', 'id', 'reference', '@version', 'result', 'meta', 'schedule', 'pscheduler'])

In [19]:
uuid = bbrmon_pscheduler_documents[0]['_source']['id']#.keys()

'8b0cf698-6144-4dc1-8090-c246ba5c3bdc'

In [22]:
hostname = bbrmon_pscheduler_documents[0]['_source']['meta']['hostname']#.keys()

{'observer': {'ip': '127.0.0.1', 'hostname': 'localhost', 'geo': {'as': {}}}}

In [25]:
bbrmon_pscheduler_documents[0]['_source']['pscheduler'].keys()

dict_keys(['added', 'end_time', 'task_href', 'run_href', 'test_checksum', 'start_time', 'task_id', 'participants', 'tool', 'run_id', 'duration'])

In [36]:
bbrmon_pscheduler_documents[0]['_source']['pscheduler']['duration']#.keys()

6.0

In [38]:
bbrmon_ss_result = es.search(index=bbrmon_ss[0],
                            body={"query":{"match_all":{}}},
                            size=10000,
                            )
bbrmon_ss_documents = [doc for doc in bbrmon_ss_result['hits']['hits']]

bbrmon_ss_documents[0].keys()

TransportError: TransportError(503, 'search_phase_execution_exception')